In [1]:
import json
import os
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from tqdm import tqdm

In [2]:
# Define the image transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [3]:
class ImageDataset(Dataset):
    def __init__(self, image_dir, info_file, transform=None):
        self.image_dir = image_dir
        self.image_paths = []
        self.labels = []
        self.label_to_idx = {}
        
        with open(info_file, 'r') as f:
            info = json.load(f)
        
        # Create a mapping from labels to integers
        unique_labels = set(info.values())
        for idx, label in enumerate(unique_labels):
            self.label_to_idx[label] = idx
        
        for image_path, label in info.items():
            self.image_paths.append(os.path.join(image_dir, image_path))
            self.labels.append(self.label_to_idx[label])  # Convert label to integer using the mapping
        
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        try:
            image = Image.open(image_path).convert('RGB')
        except FileNotFoundError:
            print(f"Error: File not found: {image_path}")
            # Return a default value or handle the missing file appropriately
            return torch.zeros(3, 224, 224), 0  # Replace with your desired default value

        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        # image = image.unsqueeze(0)  # Add a batch dimension
        return image, label

#     def __getitem__(self, idx):
#         image_path = self.image_paths[idx]
#         image = Image.open(image_path).convert('RGB')
#         if self.transform:
#             image = self.transform(image)
#         label = self.labels[idx]
#         # image = image.unsqueeze(0)  # Add a batch dimension
#         return image, label


In [4]:
# Define the contrastive loss function
class ContrastiveLoss(nn.Module):
    def __init__(self, margin=1.0):
        super(ContrastiveLoss, self).__init__()
        self.margin = margin

    def forward(self, x1, x2, y):
        distances = torch.sqrt(((x1 - x2) ** 2).sum(dim=1))
        losses = y * distances ** 2 + (1 - y) * torch.clamp(self.margin - distances, min=0.0) ** 2
        return losses.mean()

In [5]:
# Define the image similarity model
class ImageSimilarityModel(nn.Module):
    def __init__(self):
        super(ImageSimilarityModel, self).__init__()
        self.backbone = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(128, 256, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(256, 512, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(512 * 1 * 1, 128)
    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x

In [6]:


# Load the dataset
train_dataset = ImageDataset('train/', 'train_image_info.json', transform=transform)
query_dataset = ImageDataset('query_images/', 'test_image_info.json', transform=transform)
gallery_dataset = ImageDataset('gallery/', 'test_image_info.json', transform=transform)

In [7]:
train_dataset

In [8]:

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
query_loader = DataLoader(query_dataset, batch_size=1, shuffle=False)
gallery_loader = DataLoader(gallery_dataset, batch_size=1, shuffle=False)

In [9]:
# Initialize the model and optimizer
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = ImageSimilarityModel().to(device)
criterion = ContrastiveLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [10]:
num_epochs = 30
for epoch in range(num_epochs):
    running_loss = 0.0
    for images, labels in tqdm(train_loader, desc=f'Epoch {epoch+1}/{num_epochs}'):
        images = images.to(device)
        labels = [label.to(device) for label in labels]

        optimizer.zero_grad()
        embeddings = model(images)

        # Create matching and non-matching pairs
        batch_size = embeddings.size(0)
        y = torch.eye(batch_size).to(device)  # Create a batch-wise identity matrix
        y = y.view(-1)  # Flatten the matrix to create matching/non-matching pairs

        anchor_embeddings = embeddings.repeat(batch_size, 1)  # Repeat anchor embeddings
        positive_embeddings = embeddings.repeat(1, batch_size).view(-1, embeddings.size(-1))  # Repeat positive embeddings

        loss = criterion(anchor_embeddings, positive_embeddings, y)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f'Epoch {epoch+1}/{num_epochs}, Loss: {running_loss / len(train_loader)}')

Epoch 1/30: 100%|█████████████████████████████| 358/358 [03:34<00:00,  1.67it/s]


Epoch 1/30, Loss: nan


Epoch 2/30: 100%|█████████████████████████████| 358/358 [04:07<00:00,  1.45it/s]


Epoch 2/30, Loss: nan


Epoch 3/30: 100%|█████████████████████████████| 358/358 [04:43<00:00,  1.26it/s]


...

Epoch 28/30, Loss: nan


Epoch 29/30: 100%|████████████████████████████| 358/358 [04:09<00:00,  1.43it/s]


Epoch 29/30, Loss: nan


Epoch 30/30: 100%|████████████████████████████| 358/358 [04:06<00:00,  1.45it/s]

Epoch 30/30, Loss: nan


In [11]:
query_embeddings = []
gallery_embeddings = []

with torch.no_grad():
    for query_image, _ in tqdm(query_loader, desc='Computing query embeddings'):
        query_image = query_image.to(device)
        embedding = model(query_image)
        query_embeddings.append(embedding)

    for gallery_image, _ in tqdm(gallery_loader, desc='Computing gallery embeddings'):
        gallery_image = gallery_image.to(device)
        embedding = model(gallery_image)
        gallery_embeddings.append(embedding)

query_embeddings = torch.cat(query_embeddings, dim=0)
gallery_embeddings = torch.cat(gallery_embeddings, dim=0)

Computing query embeddings:   1%|▏           | 13/1150 [00:00<00:08, 126.45it/s]

Error: File not found: query_images/Image_10506.jpg
Error: File not found: query_images/Image_4904.jpg
Error: File not found: query_images/Image_4080.jpg
Error: File not found: query_images/Image_9516.jpg
Error: File not found: query_images/Image_2124.jpg
Error: File not found: query_images/Image_5555.jpg
Error: File not found: query_images/Image_11382.jpg
Error: File not found: query_images/Image_10259.jpg
Error: File not found: query_images/Image_3842.jpg
Error: File not found: query_images/Image_5510.jpg
Error: File not found: query_images/Image_9721.jpg
Error: File not found: query_images/Image_9186.jpg
Error: File not found: query_images/Image_5552.jpg
Error: File not found: query_images/Image_5657.jpg
Error: File not found: query_images/Image_3789.jpg
Error: File not found: query_images/Image_3178.jpg
Error: File not found: query_images/Image_5829.jpg
Error: File not found: query_images/Image_5570.jpg
Error: File not found: query_images/Image_9366.jpg
Error: File not found: query

Computing query embeddings:   2%|▎           | 28/1150 [00:00<00:08, 139.81it/s]

Error: File not found: query_images/Image_8684.jpg
Error: File not found: query_images/Image_10406.jpg
Error: File not found: query_images/Image_1364.jpg
Error: File not found: query_images/Image_11077.jpg
Error: File not found: query_images/Image_3779.jpg
Error: File not found: query_images/Image_7523.jpg
Error: File not found: query_images/Image_10020.jpg


Computing query embeddings:   4%|▍           | 43/1150 [00:00<00:07, 139.58it/s]

...

Error: File not found: gallery/Image_1052.jpg
Error: File not found: gallery/Image_3076.jpg
Error: File not found: gallery/Image_7719.jpg
Error: File not found: gallery/Image_507.jpg
Error: File not found: gallery/Image_2877.jpg
Error: File not found: gallery/Image_959.jpg
Error: File not found: gallery/Image_1668.jpg
Error: File not found: gallery/Image_12213.jpg
Error: File not found: gallery/Image_7100.jpg
Error: File not found: gallery/Image_4855.jpg
Error: File not found: gallery/Image_10945.jpg
Error: File not found: gallery/Image_10340.jpg
Error: File not found: gallery/Image_10245.jpg
Error: File not found: gallery/Image_11414.jpg
Error: File not found: gallery/Image_11549.jpg
Error: File not found: gallery/Image_516.jpg
Error: File not found: gallery/Image_1327.jpg
Error: File not found: gallery/Image_8385.jpg
Error: File not found: gallery/Image_495.jpg
Error: File not found: gallery/Image_2624.jpg
Error: File not found: gallery/Image_498.jpg
Error: File not found: gallery/Im

Computing gallery embeddings:  98%|███████▊| 1126/1150 [00:09<00:00, 168.58it/s]

Error: File not found: gallery/Image_6551.jpg
Error: File not found: gallery/Image_3613.jpg
Error: File not found: gallery/Image_4379.jpg
Error: File not found: gallery/Image_3437.jpg
Error: File not found: gallery/Image_7923.jpg
Error: File not found: gallery/Image_10587.jpg
Error: File not found: gallery/Image_10454.jpg
Error: File not found: gallery/Image_6714.jpg
Error: File not found: gallery/Image_4680.jpg
Error: File not found: gallery/Image_5258.jpg
Error: File not found: gallery/Image_3959.jpg
Error: File not found: gallery/Image_8572.jpg
Error: File not found: gallery/Image_12539.jpg
Error: File not found: gallery/Image_4759.jpg
Error: File not found: gallery/Image_11300.jpg
Error: File not found: gallery/Image_5632.jpg
Error: File not found: gallery/Image_12454.jpg
Error: File not found: gallery/Image_7338.jpg
Error: File not found: gallery/Image_9694.jpg
Error: File not found: gallery/Image_8608.jpg
Error: File not found: gallery/Image_7461.jpg
Error: File not found: galler

Computing gallery embeddings: 100%|████████| 1150/1150 [00:09<00:00, 116.03it/s]

Error: File not found: gallery/Image_10017.jpg
Error: File not found: gallery/Image_6115.jpg
Error: File not found: gallery/Image_852.jpg
Error: File not found: gallery/Image_2049.jpg
Error: File not found: gallery/Image_10457.jpg
Error: File not found: gallery/Image_1344.jpg
Error: File not found: gallery/Image_12295.jpg
Error: File not found: gallery/Image_3880.jpg
Error: File not found: gallery/Image_9129.jpg
Error: File not found: gallery/Image_1130.jpg
Error: File not found: gallery/Image_3897.jpg


In [12]:
# Compute similarity scores and rank the gallery images
ranks = []
for query_embedding in tqdm(query_embeddings, desc='Ranking gallery images'):
    similarities = torch.mm(gallery_embeddings, query_embedding.unsqueeze(1)).squeeze()
    sorted_indices = torch.argsort(similarities, descending=True)
    ranks.append(sorted_indices)

Ranking gallery images: 100%|████████████| 1150/1150 [00:00<00:00, 16256.72it/s]


In [16]:
ranks

[tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1149]),
 tensor([   0,    1,    2,  ..., 1147, 1148, 1

In [13]:
# Compute evaluation metrics
mAP_1 = 0.0
mAP_10 = 0.0
mAP_50 = 0.0
mean_rank = 0.0

for i, rank in enumerate(ranks):
    query_label = query_dataset.labels[i]
    gallery_labels = [gallery_dataset.labels[idx.item()] for idx in rank]

    # Compute mAP@1
    if gallery_labels[0] == query_label:
        mAP_1 += 1.0

    # Compute mAP@10
    precision_at_10 = sum(1 for label in gallery_labels[:10] if label == query_label) / 10
    mAP_10 += precision_at_10

    # Compute mAP@50
    precision_at_50 = sum(1 for label in gallery_labels[:50] if label == query_label) / 50
    mAP_50 += precision_at_50

    # Compute mean rank
    rank_of_first_match = next((i for i, label in enumerate(gallery_labels) if label == query_label), len(gallery_labels))
    mean_rank += rank_of_first_match

      
mAP_1 /= len(ranks)
mAP_10 /= len(ranks)
mAP_50 /= len(ranks)
mean_rank /= len(ranks)

print(f'mAP@1: {mAP_1}')
print(f'mAP@10: {mAP_10}')
print(f'mAP@50: {mAP_50}')
print(f'Mean Rank: {mean_rank}')

mAP@1: 0.06695652173913043
mAP@10: 0.06695652173913043
mAP@50: 0.06695652173913043
Mean Rank: 466.88260869565215
